In [1]:
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
from more_itertools import locate
from datetime import datetime
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from support.pd_support import *

In [ ]:
def read_rigid_body_csv(_pth):
    """
    _pth: path to the rigid body file
    """
    df = pd.read_csv(_pth, skiprows=2, header=None)

    # get the start time of the capture
    raw_df = pd.read_csv(_pth)
    cols_list = raw_df.columns     # first row which contains capture start time
    inx = [i for i, x in enumerate(cols_list) if x == "Capture Start Time"]
    st_time = cols_list[inx[0] + 1]
    st_time = datetime.strptime(st_time, "%Y-%m-%d %I.%M.%S.%f %p")  # returns datetime object

    _marker_type = []
    for idx in df.columns:
        _marker_type.append(df[idx][0])

    def find_indices(list_to_check, item_to_find):
        indices = locate(list_to_check, lambda x: x == item_to_find)
        return list(indices)

    # find the indices of the marker types
    _rb_idx = find_indices(_marker_type, "Rigid Body")
    _rb_marker_idx = find_indices(_marker_type, "Rigid Body Marker")
    _marker_idx = find_indices(_marker_type, "Marker")

    _rb_idx.sort()
    _rb_marker_idx.sort()
    _marker_idx.sort()

    _rb_df = df.copy(deep=True)
    _rb_df = df[1:]
    _analysis_type = list(_rb_df.iloc[2].values)

    _rotation_ids = find_indices(_analysis_type, "Rotation")
    _rotation_ids.sort()

    _rb_pos_idx = _rb_idx.copy()

    # remove the rotation indices from the position indices
    [_rb_pos_idx.remove(i) for i in _rotation_ids]

    # create column names for angle
    col_names = []

    # first two columns
    col_names.append("frame")
    col_names.append("time_seconds")

    for i in _rotation_ids:
        _col = _rb_df[i].iloc[3].lower()
        col_names.append("rb_ang_" + _col)

    # create column names for position
    for i in _rb_pos_idx:
        if isinstance(_rb_df[i].iloc[3], str):
            _col = _rb_df[i].iloc[3].lower()
            col_names.append("rb_pos_" + _col)
        else:
            col_names.append("rb_pos_err")


    # rigid body individual marker section

    for i in _rb_marker_idx:
        if isinstance(_rb_df[i].iloc[0], str):
            _col_head = _rb_df[i].iloc[0].lower()
            _col_head = _col_head.split(":")[1].strip()
            _col_head = _col_head.replace("marker", "")
            _m_idx = int(_col_head)
            
            if isinstance(_rb_df[i].iloc[3], str):
                _col = _rb_df[i].iloc[3].lower()
                col_names.append("rb_marker_m" + str(_col_head) + "_" + _col)
            else:
                col_names.append("rb_marker_m" + str(_col_head) + "_mq") # marker quality


    # individual marker section 

    for i in _marker_idx:
        if isinstance(_rb_df[i].iloc[0], str):
            _col_head = _rb_df[i].iloc[0].lower()
            _col_head = _col_head.split(":")[1].strip()
            _col_head = _col_head.replace("marker", "")
            _m_idx = int(_col_head)
            
            if isinstance(_rb_df[i].iloc[3], str):
                _col = _rb_df[i].iloc[3].lower()
                col_names.append("m" + str(_col_head) + "_" + _col)

    _rb_df = _rb_df[4:]
    _rb_df.columns = col_names

    #reset index
    _rb_df = _rb_df.reset_index(drop=True)

    return _rb_df, st_time

In [2]:
_pth = r"C:\Users\Sujith\Documents\Projects\armbo\recording_programs\test_data\sensor_jan_13_2023\sk21\sk21_forward_t0.csv"

In [3]:
df, _st_time = read_rigid_body_csv(_pth)
df

,frame,time_seconds,rb_ang_x,rb_ang_y,rb_ang_z,rb_ang_w,rb_pos_x,rb_pos_y,rb_pos_z,rb_pos_err,...,m1_z,m2_x,m2_y,m2_z,m3_x,m3_y,m3_z,m4_x,m4_y,m4_z
0,0,0.000000,0.000092,0.000006,-0.000000,-1.000000,0.076603,0.075775,0.046148,0.000020,...,0.112039,0.105214,0.075029,0.008475,0.081736,0.076534,0.057764,0.062593,0.075593,0.006313
1,1,0.010000,0.000019,-0.000044,0.000272,-1.000000,0.076602,0.075777,0.046140,0.000028,...,0.112037,0.105214,0.075027,0.008474,0.081737,0.076534,0.057763,0.062592,0.075596,0.006312
2,2,0.020000,0.000020,-0.000016,-0.000006,-1.000000,0.076606,0.075773,0.046139,0.000011,...,0.112034,0.105215,0.075026,0.008474,0.081738,0.076534,0.057762,0.062593,0.075598,0.006312
3,3,0.030000,0.000070,-0.000044,0.000174,-1.000000,0.076609,0.075776,0.046145,0.000017,...,0.112033,0.105215,0.075026,0.008473,0.081739,0.076533,0.057761,0.062593,0.075599,0.006312
4,4,0.040000,0.000051,-0.000040,0.000305,-1.000000,0.076612,0.075775,0.046145,0.000023,...,0.112032,0.105215,0.075025,0.008473,0.081740,0.076532,0.057761,0.062594,0.075600,0.006313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,2402,24.020000,0.000346,-0.022780,0.000264,-0.999740,0.022805,0.076284,-0.068162,0.000088,...,-0.001500,0.049736,0.075576,-0.107067,0.028436,0.076938,-0.056857,0.006936,0.076091,-0.107216
2403,2403,24.030000,0.000393,-0.022833,0.000058,-0.999739,0.022801,0.076292,-0.068161,0.000085,...,-0.001499,0.049736,0.075576,-0.107067,0.028436,0.076939,-0.056856,0.006935,0.076091,-0.107215
2404,2404,24.040000,0.000358,-0.022802,0.000029,-0.999740,0.022799,0.076290,-0.068161,0.000085,...,-0.001499,0.049736,0.075576,-0.107066,0.028436,0.076942,-0.056854,0.006935,0.076091,-0.107215
2405,2405,24.050000,0.000381,-0.022783,0.000146,-0.999740,0.022799,0.076286,-0.068162,0.000082,...,-0.001498,0.049737,0.075576,-0.107065,0.028436,0.076945,-0.056852,0.006934,0.076092,-0.107215


In [ ]:
_rb_df.iloc[0]